<a href="https://cognitiveclass.ai"><img src = "https://ibm.box.com/shared/static/9gegpsmnsoo25ikkbl4qzlvlyjbgxs5x.png" width = 400> </a>

<h1 align=center><font size = 5>Capstone Project - The Battle of Neighborhoods (Week 1)</font></h1>

## Table of Contents

1. <a href="#item1">A description of the problem and a discussion of the background.</a>
2. <a href="#item2">A description of the data and how it will be used to solve the problem.</a>  

<h3>Instructions 1</h3>

Clearly define a problem or an idea of your choice, where you would need to leverage the Foursquare location data to solve or execute. Remember that <br/>data science problems always target an audience and are meant to help a group of stakeholders solve a problem, so make sure that you explicitly<br/> describe your audience and why they would care about your problem.

## 1. A description of the problem and a discussion of the background

<h2>Introduction/Business Problem</h2><br/>
Introduction where you discuss the business problem and who would be interested in this project.

<h3>1.1 Description of the Problem</h3>
<p>The population of London has grown considerably over the last decades. London is very diverse. It represents what is called the reflection of the old British Empire. In London, you can get fresh from food supplies from Africa. One begins to wonder the efficiency of the supply mechanism.</p>

<p>The real deal is that as much as there are many fine restaurants in London – Asian, Middle Eastern, Latin and American restaurants, you can struggle to find good place to dine in the finest of West African cuisine that has combination of Nigerian, Ghanaian, Cameroonian, Senegalese and more.</p>

<h3>1.2 Discussion of the Background</h3>
<p>My client, a successful restaurant chain in Africa is looking to expand operation into Europe through London. They want to create a high-end restaurant that comes with organic mix and healthy. Their target is not only West Africans, but they are pro-organic and healthy eating. To them every meal counts and counts as a royal when you eat.</p>

<p>Since the London demography is so big, my client needs deeper insight from available data in other to decide where to establish the first Europe “palace” restaurant. This company spends a lot on research and provides customers with data insight into the ingredients used at restaurants.</p>

<h3>1.3 Target Audience</h3>
<p>Considering the diversity of London, there is a high multicultural sense. London is a place where different shades live. As such, in the search for an high-end African-inclined restaurant, there is a high shortage.</p>

<h3>Instructions 2</h3>

Describe the data that you will be using to solve the problem or execute your idea. Remember that you will need to use the Foursquare location data to<br/> solve the problem or execute your idea. You can absolutely use other datasets in combination with<br/> the Foursquare location data. So make sure that you provide adequate explanation and discussion, with examples, of the data <br/>that you will be using, even if it is only Foursquare location data.

## 2. A description of the data and a how it will be used to solve the problem

<h3>2.1 Description of the Data</h3><br/>
This project will rely on public data from Wikipedia and Foursquare.<br/>

### a. Frist dataset¶
In this project, London will be used as synonymous to the "Greater London Area" in this project. Within the Greater London Area, there are areas that are within the London Area Postcode. The focus of this project will be the nieghbourhoods are that are within the London Post Code area.<br/>

The London Area consists of 32 Boroughs and the "City of London". Our data will be from the link - Greater London Area https://en.wikipedia.org/wiki/List_of_areas_of_London<br/>

A sample of the web scrapped of the Wikipedia page for the Greater London Area data is provided below :

In [27]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [28]:
wikipedia_link = 'https://en.wikipedia.org/wiki/List_of_areas_of_London'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:64.0) Gecko/20100101 Firefox/64.0'}
wikipedia_page = requests.get(wikipedia_link, headers = headers)
wikipedia_page

<Response [200]>

In [29]:
# Cleans html file
soup = BeautifulSoup(wikipedia_page.content, 'html.parser')
# This extracts the "tbody" within the table where class is "wikitable sortable"
table = soup.find('table', {'class':'wikitable sortable'}).tbody
# table

In [30]:
# Extracts all "tr" (table rows) within the table above
rows = table.find_all('tr')
# rows

In [31]:
# Extracts the column headers, removes and replaces possible '\n' with space for the "th" tag
columns = [i.text.replace('\n', '')
           for i in rows[0].find_all('th')]
# columns

In [32]:
# Converts columns to pd dataframe
df = pd.DataFrame(columns = columns)
# df

In [33]:
# Extracts every row with corresponding columns
# Then appends the values to the create pd dataframe "df"
# Please not that the first row (row[0]) is skipped because it is already the header

for i in range(1, len(rows)):
    tds = rows[i].find_all('td')
    
    
    if len(tds) == 7:
        values = [tds[0].text, tds[1].text, tds[2].text.replace('\n', ''.replace('\xa0','')), tds[3].text, tds[4].text.replace('\n', ''.replace('\xa0','')), tds[5].text.replace('\n', ''.replace('\xa0','')), tds[6].text.replace('\n', ''.replace('\xa0',''))]
    else:
        values = [td.text.replace('\n', '').replace('\xa0','') for td in tds]
        
        df = df.append(pd.Series(values, index = columns), ignore_index = True)

        df

In [34]:
df.head()

,Location,London borough,Post town,Postcode district,Dial code,OS grid ref
0,Abbey Wood,"Bexley, Greenwich [7]",LONDON,SE2,020,TQ465785
1,Acton,"Ealing, Hammersmith and Fulham[8]",LONDON,"W3, W4",020,TQ205805
2,Addington,Croydon[8],CROYDON,CR0,020,TQ375645
3,Addiscombe,Croydon[8],CROYDON,CR0,020,TQ345665
4,Albany Park,Bexley,"BEXLEY, SIDCUP","DA5, DA14",020,TQ478728


In [35]:
df = df.rename(index=str, columns = {'Location': 'Location', 'London\xa0borough': 'Borough', 'Post town': 'Post-town', 'Postcode\xa0district': 'Postcode', 'Dial\xa0code': 'Dial-code', 'OS grid ref': 'OSGridRef'})

In [36]:
df['Borough'] = df['Borough'].map(lambda x: x.rstrip(']').rstrip('0123456789').rstrip('['))

In [37]:
df0 = df.drop('Postcode', axis=1).join(df['Postcode'].str.split(',', expand=True).stack().reset_index(level=1, drop=True).rename('Postcode'))

In [38]:
df1 = df0[['Location', 'Borough', 'Postcode', 'Post-town']].reset_index(drop=True)

In [39]:
df1.head()

,Location,Borough,Postcode,Post-town
0,Abbey Wood,"Bexley, Greenwich",SE2,LONDON
1,Acton,"Ealing, Hammersmith and Fulham",W3,LONDON
2,Acton,"Ealing, Hammersmith and Fulham",W4,LONDON
3,Angel,Islington,EC1,LONDON
4,Angel,Islington,N1,LONDON


In [40]:
df2 = df1

In [41]:
# df_toronto_loc.Borough.str.contains('Toronto') -> TRUE or FALSE output
df21 = df2[df2['Post-town'].str.contains('LONDON')]

In [42]:
df21.head(10)

,Location,Borough,Postcode,Post-town
0,Abbey Wood,"Bexley, Greenwich",SE2,LONDON
1,Acton,"Ealing, Hammersmith and Fulham",W3,LONDON
2,Acton,"Ealing, Hammersmith and Fulham",W4,LONDON
3,Angel,Islington,EC1,LONDON
4,Angel,Islington,N1,LONDON
5,Church End,Brent,NW10,LONDON
6,Church End,Barnet,N3,LONDON
7,Clapham,"Lambeth, Wandsworth",SW4,LONDON
8,Clerkenwell,Islington,EC1,LONDON
10,Colindale,Barnet,NW9,LONDON


In [43]:
df3 = df21[['Location', 'Borough', 'Postcode']].reset_index(drop=True)

In [44]:
df_london = df3

In [45]:
!pip -q install geocoder
import geocoder

In [47]:
'''Geocoder starts here'''
'''Defining a function to use --> get_latlng()'''
def get_latlng(arcgis_geocoder):
    
    # Initialize the Location (lat. and long.) to "None"
    lat_lng_coords = None
    
    # While loop helps to create a continous run until all the location coordinates are geocoded
    
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, London, United Kingdom'.format(arcgis_geocoder))
        lat_lng_coords = g.latlng
    return lat_lng_coords
'''Geocoder ends here'''

'Geocoder ends here'

In [48]:
postal_codes = df_london['Postcode']    
coordinates = [get_latlng(postal_code) for postal_code in postal_codes.tolist()]

In [49]:
# This will store the London dataframe with coordinates
df_london_loc = df_london

# The obtained coordinates (latitude and longitude) are joined with the dataframe as shown
df_london_coordinates = pd.DataFrame(coordinates, columns = ['Latitude', 'Longitude'])
df_london_loc['Latitude'] = df_london_coordinates['Latitude']
df_london_loc['Longitude'] = df_london_coordinates['Longitude']

### b. Second dataset¶
The Foursquare API will be used to obtain the geographical location data for the London Area. These will be used to explore the venues in the neighbourhoods of London.<br/>

The venues will provide the categories needed for the analysis and eventually, these will be used to determine the viability of selected locations for the restaurant.

<h3>2.2 How data will be used to solve the problem</h3>

The data from the datasets 1 and 2 will be explored by considering the venues within the neighbourhood of London Postcode areas. <br/>
These areas' restaurants would be checked in terms of the types of restaurants within a certain mile radius. <br/>
Due to Foursquare restrictions, the number of venues will be limited to 100 venues.<br/> 
The proximity to transport connection and other amenities would be correlated. <br/>
Also, accessibility and ease of supplies of organic ingredients would be considered.